# Music EDA

## connecting to Spotify API with spotipy

In [1]:
import os
import sys
import json
from json.decoder import JSONDecodeError
import webbrowser
import spotipy
import spotipy.util as util
from pprint import pprint

import pandas as pd
import numpy as np

import fuzzyset

In [ ]:
# User ID: stevay?si=yNi65NanRZeIRXlu0Gjobw

In [2]:
username = sys.argv[1]
username

'-f'

In [3]:
try: 
    token = util.prompt_for_user_token(username,
                                       scope=None,
                                       client_id = '39cc01d4b4a544ebad4e813e7190e606',
                                       client_secret = '21e5aa26f17247bcae6d6aeba252a49f',
                                       redirect_uri='http://google.com/')
except:
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username,
                                       scope=None,
                                       client_id = '39cc01d4b4a544ebad4e813e7190e606',
                                       client_secret = '21e5aa26f17247bcae6d6aeba252a49f',
                                       redirect_uri='http://google.com/')
    

# create spotify object
spotify_object = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=39cc01d4b4a544ebad4e813e7190e606&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com%2F in your browser


Enter the URL you were redirected to: https://www.google.com/?code=AQDk8LbAdNnXBYgjTWk_cfThQ9vvvpnV-WKfSumoE5uwtOtgWIZMyfkQ2LoAsZGZWh1foEosr1i0lfs61_frIUUZD4kTrryVyEkkzeyYPj_qaduKdtHL2wsnoTX1Zl8yRFlKwSCjX0EfMrJmHVE77yBXmu2V6aQuX_m4afhZ4yFmApWog4LauG0c




## pulling USER metadata from spotify_object

In [5]:
user = spotify_object.current_user()

In [ ]:
# useful code to print JSON from spotify
# print(json.dumps(VARIABLE, sort_keys=True,indent=4))

In [6]:
print(json.dumps(user, sort_keys=True,indent=4))

{
    "display_name": "Steve Au-Yeung",
    "external_urls": {
        "spotify": "https://open.spotify.com/user/stevay"
    },
    "followers": {
        "href": null,
        "total": 9
    },
    "href": "https://api.spotify.com/v1/users/stevay",
    "id": "stevay",
    "images": [
        {
            "height": null,
            "url": "https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=10201893448121480&height=200&width=200&ext=1546041432&hash=AeSrqWrvfgHV9EEF",
            "width": null
        }
    ],
    "type": "user",
    "uri": "spotify:user:stevay"
}


# script to search for specific tracks of artists


In [496]:
# iterate through list of artists/track names in dataset
spotify_list_data = []

for index, row in df.iterrows():
    #print(row['artist'] + ' - ' + row['title'])
    data = {}
    
    # define query
    query = 'artist:' + str(row['artist'] + ' track:' + str(row['title'].replace("'",'')))
    
    #print(query)
    
    # search with Spotipy / Spotify API / spotify_object
    search_result = spotify_object.search(query, limit=5,type='track')
    
    if len(search_result['tracks']['items']) == 0:
        data['track_name'] = None
        data['track_id'] = None
        data['track_preview_url'] = None
        data['track_album_cover_art'] = None
    
    else:
        # list to store tracks from search results
        temp_list = []

        # iterate through all results to find the right track
        for i in range(len(search_result['tracks']['items'])):

            temp_list.append(search_result['tracks']['items'][i]['name'])

        # initiate new fuzzy match set
        fz = fuzzyset.FuzzySet()

        # Create a list of terms we would like to match against in a fuzzy way
        for i in temp_list:

            fz.add(i)

        # find index to pull results from Spotify
        correct_index = temp_list.index(fz.get(str(row['title']))[0][1])

        # save results
        data['track_name'] = search_result['tracks']['items'][correct_index]['name']
        data['track_id'] = search_result['tracks']['items'][correct_index]['id']
        data['track_preview_url'] = search_result['tracks']['items'][correct_index]['preview_url']
        data['track_album_cover_art'] = search_result['tracks']['items'][correct_index]['album']['images'][0]['url']
    
    # append to list
    spotify_list_data.append(data)

    
# create new DataFrame based on results from Spotify
df_2 = pd.DataFrame(spotify_list_data)

#df_2

retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [705]:
# pickle spotify track IDs
df_2.to_pickle("data/spotify_track_ids.pkl")

## Use below to load all the relevant pickle files

In [4]:
# load spotify track IDs
df_2 = pd.read_pickle("data/spotify_track_ids.pkl")

# load billboard Hot 100 tracks
df = pd.read_pickle("data/billboard_tracks.pkl")

In [707]:
df_2.head(5)

,track_album_cover_art,track_id,track_name,track_preview_url
0,https://i.scdn.co/image/9f3d46cdf43861648c8729...,3k5ycyXX5qsCjLd7R2vphp,Mona Lisa,None
1,https://i.scdn.co/image/016deabaaffedd0dbe751d...,6vSKLIxqiYllqDd2VcGVHv,I Wanna Be Loved - Single Version,None
2,https://i.scdn.co/image/9686042177407e311bdb2f...,0788TdvEjGtjFVb6CTFdJH,Tennessee Waltz,None
3,https://i.scdn.co/image/3d80355787858046e3c006...,1v52YkW75eN4RJ2EyDdQVI,I'll Never Be Free,https://p.scdn.co/mp3-preview/4e8a4b9f6a309615...
4,https://i.scdn.co/image/0557d5dbfefd07a4abdc38...,2BNPXyUhq5WFpAO1Ff5yEg,All My Love,None


# Script to get spotify audio features per track

In [529]:
# grab 50 tracks at a time
list_of_ids = []


for i in range(0,len(df_2),50):
    if i == 4000:
        list_of_ids = list_of_ids + df_2.track_id[i:4029].values.tolist()
    else:
        
        list_of_ids = list_of_ids + df_2.track_id[i:(i+50)].values.tolist()


#test_list = df_2.track_id[0:50].values.tolist()

In [525]:
test_list = []

In [527]:
test_list.append(df_2.track_id[0:50].values.tolist())

# Manually updating df_2 with spotify data

In [5]:
len(df_2.loc[df_2['track_name'].isna()]) # 33 songs not on Spotify

33

In [709]:
df.iloc[78,:]

artist                                                Nervous Norvus
difficult_words                                                   50
f_k_grade                                                        4.1
flesch_index                                                   80.28
fog_index                                                        5.2
lyrics             TRANSFUSION\nNervous Norvus\n\nZZZZZZOOOOOOOOO...
num_dupes                                                         20
num_lines                                                         40
num_syllables                                                  479.7
num_words                                                        345
pos                                                               47
sentiment          {'neg': 0.057, 'neu': 0.914, 'pos': 0.029, 'co...
tags                                            [death by cirrhosis]
title                                                    Transfusion
year                              

In [710]:
df_2.iloc[91,:]

track_album_cover_art    None
track_id                 None
track_name               None
track_preview_url        None
Name: 91, dtype: object

In [696]:
# update df_2

update_track(180,0)

In [120]:
# helper function to update df_2 manually
def update_track(row,index):
    '''
    parameters: 
    row = which row in df_2 to update
    index = the specific item in the Spotify API search result to update df_2 with
    '''

    # get the track name
    df_2.iloc[row,2] = search_results_test['tracks']['items'][index]['name']

    # get the track ID
    df_2.iloc[row,1] = search_results_test['tracks']['items'][index]['id']

    # preview url
    df_2.iloc[row,3] = search_results_test['tracks']['items'][index]['preview_url']

    # album cover art
    df_2.iloc[row,0] = search_results_test['tracks']['items'][index]['album']['images'][0]['url']

In [859]:
track_id = '0KnD456yC5JuweN932Ems3'

# NOTE: can pull 50 tracks per request

track_features = spotify_object.audio_features(tracks=[track_id])

#print(json.dumps(track_features, sort_keys=True,indent=4))

#df_3 = pd.DataFrame(track_features)
track_features

[{'danceability': 0.697,
  'energy': 0.182,
  'key': 7,
  'loudness': -12.586,
  'mode': 1,
  'speechiness': 0.0357,
  'acousticness': 0.78,
  'instrumentalness': 0,
  'liveness': 0.126,
  'valence': 0.474,
  'tempo': 82.184,
  'type': 'audio_features',
  'id': '0KnD456yC5JuweN932Ems3',
  'uri': 'spotify:track:0KnD456yC5JuweN932Ems3',
  'track_href': 'https://api.spotify.com/v1/tracks/0KnD456yC5JuweN932Ems3',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0KnD456yC5JuweN932Ems3',
  'duration_ms': 158000,
  'time_signature': 3}]

# Use below to search track content in Spotify

In [ ]:
# use 98\u00ba for 98 Degrees...

In [692]:
artist_name = 'marty robbins'
track_name = 'the story of my life'

In [693]:
query = "artist:" + artist_name + " track:" + track_name # define filters
search_results_test = spotify_object.search(query,limit=5,type='track')

In [694]:
len(search_results_test['tracks']['items'])

5

In [695]:
print(json.dumps(search_results_test,sort_keys=True,indent=4))

{
    "tracks": {
        "href": "https://api.spotify.com/v1/search?query=artist%3Amarty+robbins+track%3Athe+story+of+my+life&type=track&market=US&offset=0&limit=5",
        "items": [
            {
                "album": {
                    "album_type": "album",
                    "artists": [
                        {
                            "external_urls": {
                                "spotify": "https://open.spotify.com/artist/0Xi59sEw38vRvwleSAVqoo"
                            },
                            "href": "https://api.spotify.com/v1/artists/0Xi59sEw38vRvwleSAVqoo",
                            "id": "0Xi59sEw38vRvwleSAVqoo",
                            "name": "Marty Robbins",
                            "type": "artist",
                            "uri": "spotify:artist:0Xi59sEw38vRvwleSAVqoo"
                        }
                    ],
                    "available_markets": [
                        "AD",
                        "AE",
          

In [691]:
# get the track name

search_results_test['tracks']['items'][0]['name']

'The Cowboy In the Continental Suit'

In [552]:
# get the track ID

search_results_test['tracks']['items'][0]['id']

'4P0As0E8QfskHuiFTGQHHl'

In [553]:
# preview url

search_results_test['tracks']['items'][0]['preview_url']

'https://p.scdn.co/mp3-preview/2cb0920c5bb3a08ed8dca7f930bba74daa1f10d7?cid=39cc01d4b4a544ebad4e813e7190e606'

In [554]:
# album cover art

search_results_test['tracks']['items'][0]['album']['images'][0]['url']

'https://i.scdn.co/image/9a2fc67e04993d8d14a5c441ecb5d45bb365536b'

# Helper function to pretty print track audio features

In [865]:
def get_audio_features(spotify, tracks, pretty_print=False):
    """
    Given a list of tracks, get and print the audio features for those tracks!
    :param spotify: An authenticated Spotipy instance
    :param tracks: A list of track dictionaries
    """
    if not tracks:
        print('No tracks provided.')
        return

    # Build a map of id->track so we can get the full track info later
    track_map = {track.get('id'): track for track in tracks}

    # Request the audio features for the chosen tracks (limited to 50)
    
    tracks_features_response = spotify_object.audio_features(tracks=track_map.keys())
    track_features_map = {f.get('id'): f for f in tracks_features_response}

    # Iterate through the features and print the track and info
    if pretty_print:
        for track_id, track_features in track_features_map.items():
            # Print out the track info and audio features
            track = track_map.get(track_id)
            print_audio_features_for_track(track, track_features)
            

    return track_features_map

## artist details

In [11]:
artist = search_results['artists']['items'][0]
artist

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
 'followers': {'href': None, 'total': 3671769},
 'genres': ['alternative rock',
  'art rock',
  'melancholia',
  'modern rock',
  'permanent wave',
  'rock'],
 'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
 'id': '4Z8W4fKeB5YxbusRsdQVPb',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/afcd616e1ef2d2786f47b3b4a8a6aeea24a72adc',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/563754af10b3d9f9f62a3458e699f58c4a02870f',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/4067ea225d8b42fa6951857d3af27dd07d60f3c6',
   'width': 160}],
 'name': 'Radiohead',
 'popularity': 80,
 'type': 'artist',
 'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}

In [12]:
artist_name = artist['name']
artist_id = artist['id']
artist_followers = artist['followers']['total']
artist_genres = artist['genres']

In [13]:
# open an image found under 'images'
webbrowser.open(artist['images'][0]['url'])

True

In [14]:
# album and track details
track_uri = []
track_art = []
z = 0

# Extract albums for a specific artist

In [15]:
# extract album data
album_results = spotify_object.artist_albums(artist_id,album_type='album')

In [16]:
print(json.dumps(album_results, sort_keys=True,indent=4))

{
    "href": "https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb/albums?offset=0&limit=20&include_groups=album",
    "items": [
        {
            "album_group": "album",
            "album_type": "album",
            "artists": [
                {
                    "external_urls": {
                        "spotify": "https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb"
                    },
                    "href": "https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb",
                    "id": "4Z8W4fKeB5YxbusRsdQVPb",
                    "name": "Radiohead",
                    "type": "artist",
                    "uri": "spotify:artist:4Z8W4fKeB5YxbusRsdQVPb"
                }
            ],
            "available_markets": [
                "AD",
                "AE",
                "AR",
                "AT",
                "AU",
                "BE",
                "BG",
                "BH",
                "BO",
                "BR",
           

In [41]:
album = album_results['items'][4] # in rainbows album details

In [44]:
album_id = album['id']

# Extract tracks for a specific album

In [225]:
track_results = spotify_object.album_tracks('5qWqMyXJ9uOCl6vEIVKqBD')

In [226]:
print(json.dumps(track_results, sort_keys=True,indent=4))

{
    "href": "https://api.spotify.com/v1/albums/5qWqMyXJ9uOCl6vEIVKqBD/tracks?offset=0&limit=50",
    "items": [
        {
            "artists": [
                {
                    "external_urls": {
                        "spotify": "https://open.spotify.com/artist/7u6FSuqT3SWE2kO8rNVjfp"
                    },
                    "href": "https://api.spotify.com/v1/artists/7u6FSuqT3SWE2kO8rNVjfp",
                    "id": "7u6FSuqT3SWE2kO8rNVjfp",
                    "name": "Tennessee Ernie Ford",
                    "type": "artist",
                    "uri": "spotify:artist:7u6FSuqT3SWE2kO8rNVjfp"
                }
            ],
            "available_markets": [
                "AD",
                "AE",
                "AR",
                "AT",
                "AU",
                "BE",
                "BG",
                "BH",
                "BO",
                "BR",
                "CA",
                "CH",
                "CL",
                "CO",
     

In [240]:
track_results['items'][6]

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7rpf45DdbpweG51Acze45i'},
   'href': 'https://api.spotify.com/v1/artists/7rpf45DdbpweG51Acze45i',
   'id': '7rpf45DdbpweG51Acze45i',
   'name': 'Kay Starr',
   'type': 'artist',
   'uri': 'spotify:artist:7rpf45DdbpweG51Acze45i'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/7u6FSuqT3SWE2kO8rNVjfp'},
   'href': 'https://api.spotify.com/v1/artists/7u6FSuqT3SWE2kO8rNVjfp',
   'id': '7u6FSuqT3SWE2kO8rNVjfp',
   'name': 'Tennessee Ernie Ford',
   'type': 'artist',
   'uri': 'spotify:artist:7u6FSuqT3SWE2kO8rNVjfp'}],
 'available_markets': ['AD',
  'AE',
  'AR',
  'AT',
  'AU',
  'BE',
  'BG',
  'BH',
  'BO',
  'BR',
  'CA',
  'CH',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DE',
  'DK',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FR',
  'GB',
  'GR',
  'GT',
  'HK',
  'HN',
  'HU',
  'ID',
  'IE',
  'IL',
  'IS',
  'IT',
  'JO',
  'JP',
  'KW',
  'LB',
  'LI',
  'LT',
  'LU',
  'LV'

In [228]:
track_name = track_results['items'][6]['name']
track_id = track_results['items'][6]['id']

In [229]:
print('track name: ' + str(track_name))
print('track ID: ' + str(track_id))

track name: I'll Never Be Free
track ID: 0KnD456yC5JuweN932Ems3


# Extract audio features for specific track(s)

In [230]:
track_id = '0KnD456yC5JuweN932Ems3'

In [231]:
# NOTE: can pull 50 tracks per request

track_features = spotify_object.audio_features(tracks=[track_id])

In [232]:
print(json.dumps(track_features, sort_keys=True,indent=4))

[
    {
        "acousticness": 0.78,
        "analysis_url": "https://api.spotify.com/v1/audio-analysis/0KnD456yC5JuweN932Ems3",
        "danceability": 0.697,
        "duration_ms": 158000,
        "energy": 0.182,
        "id": "0KnD456yC5JuweN932Ems3",
        "instrumentalness": 0,
        "key": 7,
        "liveness": 0.126,
        "loudness": -12.586,
        "mode": 1,
        "speechiness": 0.0357,
        "tempo": 82.184,
        "time_signature": 3,
        "track_href": "https://api.spotify.com/v1/tracks/0KnD456yC5JuweN932Ems3",
        "type": "audio_features",
        "uri": "spotify:track:0KnD456yC5JuweN932Ems3",
        "valence": 0.474
    }
]


In [95]:
df_test = pd.DataFrame(track_features)

In [96]:
df_test

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.0718,https://api.spotify.com/v1/audio-analysis/2dh8...,0.418,257480,0.383,2dh8dLyxdjmStWOoTkZZ3T,0.0177,4,0.0896,-11.782,1,0.0257,87.773,4,https://api.spotify.com/v1/tracks/2dh8dLyxdjmS...,audio_features,spotify:track:2dh8dLyxdjmStWOoTkZZ3T,0.352
1,0.0718,https://api.spotify.com/v1/audio-analysis/5jaf...,0.418,257480,0.383,5jafMI8FLibnjkYTZ33m0c,0.0177,4,0.0896,-11.782,1,0.0257,87.773,4,https://api.spotify.com/v1/tracks/5jafMI8FLibn...,audio_features,spotify:track:5jafMI8FLibnjkYTZ33m0c,0.352


### Here’s a quick example of using Spotipy to list the names of all the albums released by the artist ‘Birdy’:

In [68]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
#spotify = spotipy.Spotify() # spotify_object

results = spotify_object.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify_object.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Beautiful Lies
Fire Within
Live In London
Birdy
Birdy
Birdy (Deluxe Version)


## PyLyrics library for finding lyrics

In [313]:
from PyLyrics import *

lyrics_data_test = PyLyrics.getLyrics('Drake','In My Feelings') #Print the lyrics directly

/Users/auyeus/anaconda3/lib/python3.6/site-packages/PyLyrics/functions.py:78: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 78 of the file /Users/auyeus/anaconda3/lib/python3.6/site-packages/PyLyrics/functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  s = BeautifulSoup(r.text)


In [315]:
lyrics_data_test_update = lyrics_data_test.replace("\n",". ")

## functions that rate the lyric complexity

In [318]:
from textstat.textstat import textstat as ts

ts.flesch_kincaid_grade(lyrics_data_test_update)

1.8

In [319]:
ts.gunning_fog(lyrics_data_test_update)

5.35

### Here’s another example showing how to get 30 second samples and cover art for the top 10 tracks for Led Zeppelin:

In [119]:
lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'

#spotify = spotipy.Spotify()
results = spotify_object.artist_top_tracks(lz_uri)

for track in results['tracks'][:10]:
    print ('track    : ' + track['name'])
    print ('audio    : ' + track['preview_url'])
    print ('cover art: ' + track['album']['images'][0]['url'])
    print ()

track    : Stairway To Heaven
audio    : https://p.scdn.co/mp3-preview/8226164717312bc411f8635580562d67e191a754?cid=39cc01d4b4a544ebad4e813e7190e606
cover art: https://i.scdn.co/image/557a6058e3de72bf37ffcd2c12dd5932276df344

track    : Immigrant Song
audio    : https://p.scdn.co/mp3-preview/8455599677a13017978dcd3f4b210937f0a16bcb?cid=39cc01d4b4a544ebad4e813e7190e606
cover art: https://i.scdn.co/image/e6e513e68c4eefdfc7bc213b35061c9c30b42dfa

track    : Whole Lotta Love
audio    : https://p.scdn.co/mp3-preview/ce11b19a4d2de9976d7626df0717d0073863909c?cid=39cc01d4b4a544ebad4e813e7190e606
cover art: https://i.scdn.co/image/5fa3a6cc1848ea743a293d2088046746d1b09608

track    : Black Dog
audio    : https://p.scdn.co/mp3-preview/9b76619fd9d563a48d38cc90ca00c3008327b52e?cid=39cc01d4b4a544ebad4e813e7190e606
cover art: https://i.scdn.co/image/557a6058e3de72bf37ffcd2c12dd5932276df344

track    : Good Times Bad Times
audio    : https://p.scdn.co/mp3-preview/c1f024eb57b569b926c8e68cab0a6056dc7d96

### Finally, here’s an example that will get the URL for an artist image given the artist’s name:

In [112]:
import spotipy
import sys

#spotify = spotipy.Spotify()

# if len(sys.argv) > 1:
#     name = ' '.join(sys.argv[1:])
# else:
#     name = 'Radiohead'

name = 'Joseph Vincent'

results = spotify_object.search(q='artist:' + name, type='artist')
items = results['artists']['items']
if len(items) > 0:
    artist = items[0]
    print (artist['name'], artist['images'][0]['url'])

Joseph Vincent https://i.scdn.co/image/0c9696c75fdda7458e01aecb83bd9380718bfce5


# Parsing JSON files related to Billboard Top 100

- go through each JSON file, and extract Billboard Top 100 data for each year, and store into a Pandas DataFrame

In [3]:
years = range(1950,2016)

with open('data/years/1950.json') as data_file:
    data = json.load(data_file)
    
df = pd.DataFrame(data)

for year in years:
    with open('data/years/' + str(year) + '.json') as data_file:
        data = json.load(data_file)
    
    #df = df.append(data,ignore_index=True,verify_integrity=True)
    if year == 1950:
        df = pd.DataFrame(data)
    else:
        df = df.append(data,ignore_index=True,verify_integrity=True)

#df

In [6]:
df.head(5)

,artist,difficult_words,f_k_grade,flesch_index,fog_index,lyrics,num_dupes,num_lines,num_syllables,num_words,pos,sentiment,tags,title,year
0,Nat King Cole,13,2.9,88.74,5.2,"Mona Lisa, Mona Lisa, men have named you\nYou'...",10,17,189.0,145,2,"{'neg': 0.155, 'neu': 0.646, 'pos': 0.199, 'co...","[american, death by lung cancer, easy listenin...",Mona Lisa,1950
1,Andrews Sisters,40,3.3,82.31,4.4,I wanna be Loved\nBy Andrews Sisters\n\nOooo-o...,32,31,270.9,189,17,"{'neg': 0.036, 'neu': 0.74, 'pos': 0.224, 'com...",[andrews sisters],I Wanna Be Loved,1950
2,Patti Page,5,2.9,88.74,5.2,I was dancing with my darling to the Tennessee...,10,16,174.6,138,18,"{'neg': 0.053, 'neu': 0.596, 'pos': 0.351, 'co...","[country, pop]",Tennessee Waltz,1950
3,Tennessee Ernie Ford,10,0.9,99.23,4.4,Each time I hold someone new\nMy arms grow col...,3,18,135.9,117,20,"{'neg': 0.167, 'neu': 0.602, 'pos': 0.231, 'co...","[death by liver failure, spiritual]",I'll Never Be Free,1950
4,Patti Page,9,6.0,69.79,6.0,"Unfortunately, we are not licensed to display ...",0,3,46.8,32,21,"{'neg': 0.07, 'neu': 0.85, 'pos': 0.079, 'comp...","[country, pop]",All My Love,1950


In [1028]:
# pickle billboard Hot 100 tracks
df.to_pickle("data/billboard_tracks.pkl")

## description of data fields in df

# Using BeautifulSoup to web scrape the Billboard Hot 100 for 2016, 2017, and 2018

# Next, we need to append / merge the Spotify API features to the DataFrame

### Features include:

- "acousticness": 0.833,
- "analysis_url": "https://api.spotify.com/v1/audio-analysis/5k7VKj1Xwy5DjO4B0PdAOb",
- "danceability": 0.537,
- "duration_ms": 255387,
- "energy": 0.342,
- "id": "5k7VKj1Xwy5DjO4B0PdAOb",
- "instrumentalness": 0.6,
- "key": 4,
- "liveness": 0.0856,
- "loudness": -9.785,
- "mode": 1,
- "speechiness": 0.0338,
- "tempo": 128.331,
- "time_signature": 3,
- "track_href": "https://api.spotify.com/v1/tracks/5k7VKj1Xwy5DjO4B0PdAOb",
- "type": "audio_features",
- "uri": "spotify:track:5k7VKj1Xwy5DjO4B0PdAOb",
- "valence": 0.156

In [ ]:
df.combine()

## Sort Genres

### get genres count of billboard hot 100 songs

In [24]:
import json
from pprint import pprint

genres = {}

# Iterate through years
for year in range(1950, 2016):
    with open('data/years/' + str(year) + '.json') as data_file:
        data = json.load(data_file)
        for song in data:
            for tag in song['tags']:
                if tag not in genres:
                    genres[tag] = 1
                else:
                    genres[tag] += 1

from operator import itemgetter
genres_sorted = sorted(genres.items(), key=itemgetter(1),reverse=True)

pprint(genres_sorted)

[('pop', 1723),
 ('classic pop and rock', 1290),
 ('rock', 1063),
 ('american', 1038),
 ('pop rock', 903),
 ('dance-pop', 531),
 ('british', 474),
 ('hip hop rnb and dance hall', 450),
 ('pop and chart', 446),
 ('uk', 435),
 ('contemporary r&b', 397),
 ('english', 395),
 ('rnb', 385),
 ('hip hop', 383),
 ('country', 375),
 ('electropop', 365),
 ('soul', 360),
 ('adult contemporary', 339),
 ('pop soul', 335),
 ('r&b', 335),
 ('pop/rock', 288),
 ('hard rock', 275),
 ('rock and indie', 243),
 ('dance', 243),
 ('pop rap', 243),
 ('soft rock', 224),
 ('usa', 198),
 ('funk', 196),
 ('folk', 196),
 ('rock and roll', 188),
 ('rhythm & blues', 186),
 ('alternative rock', 177),
 ('blues rock', 174),
 ('singer/songwriter', 173),
 ('soul and reggae', 169),
 ('jazz', 164),
 ('rock & roll', 157),
 ('américain', 157),
 ('disco', 149),
 ('hip-hop', 146),
 ('electronic', 141),
 ('blue-eyed soul', 131),
 ('easy listening soundtracks and musicals', 128),
 ('smooth soul', 128),
 ('psychedelic rock', 125),

# Music EDA Passion Project

Purpose of this project is to look at the Billboard Hot 100 songs from 1950-2015 and see how music has evolved over the last 65+ years. 

We will be using a dataset curated and shared by Kevin Schaich (github page available)

We will also use the Spotify API to get audio features (TBD on what we are using features for)

Time permitted, we will see if we can do some modeling (e.g. topic modeling)